In [ ]:
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize(project='Your account')

In [ ]:
# Define the region of interest
roi = ee.Geometry.Rectangle([115, 25, 116, 26])

In [ ]:
# Applies scaling factors.
def applyScaleFactors(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  return image.addBands(opticalBands, None, True)

# reomove cloud
def cloudRemoval(image):
  cloudShadowBitMask = (1 << 4)
  cloudsBitMask = (1 << 3)
  qa = image.select('QA_PIXEL')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).toDouble() \
              .copyProperties(image) \
              .copyProperties(image, ["system:time_start"])
# Fill the strips
def GapFill(image):
   return image.focal_mean(1,'square','pixels',12).blend(image)

# Standardize band names
LC8_BANDS = ['SR_B2','SR_B4','SR_B5','QA_PIXEL'];
LC7_BANDS = ['SR_B1','SR_B3', 'SR_B4','QA_PIXEL'];
LC5_BANDS = ['SR_B1','SR_B3', 'SR_B4','QA_PIXEL'];
STD_NAMES = ['blue', 'red', 'nir','QA_PIXEL'];

# Clip the table using a map function
def clipImage(image):
  return image.clip(roi)

# For this function to work sigma has to be previously computed (see bellow)
def addKNDVI(image,sigma):
  # Compute kernel (k) that compares NIR and red, and the normalized kernel index kNDVI
  red = image.select('red')
  nir = image.select('nir')
  D2 = nir.subtract(red).pow(2).select([0],['d2'])

  # Kndvi RBF kernel
  kndvi = D2.divide(sigma.multiply(2).pow(2)).tanh()
  return image.addBands(kndvi.select([0], ['kndvi']))

# We estimate the sigma with a naive sigma=0.5(n+r) for each pixel and time acquisition in the image collection.
# To estimate sigma using the time series we first need to compute (nir+red)*0.5 for all images in the collection
def addD2(image):
  red = image.select('red')
  nir = image.select('nir')
  sigma = nir.add(red).multiply(0.5).select([0],['sigma'])
  return image.addBands(sigma)

# Calculate monthly kNDVI
def generate_median_kndvi_collection(start_year, end_year):
    # Create an empty image collection to store the synthesized images
    median_kndvi_collection = ee.ImageCollection([])

    for year in range(start_year, end_year + 1):
        for month in range(1, 13):

            # Acquire and filter Landsat imagery.
            l8_col = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                      .filterBounds(roi)
                      .filter(ee.Filter.calendarRange(year, year, 'year'))
                      .filter(ee.Filter.calendarRange(month, month, 'month'))
                      .map(applyScaleFactors)
                      .select(LC8_BANDS, STD_NAMES)
                      .map(cloudRemoval))
            l7_col = (ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
                      .filterBounds(roi)
                      .filter(ee.Filter.calendarRange(year, year, 'year'))
                      .filter(ee.Filter.calendarRange(month, month, 'month'))
                      .map(applyScaleFactors)
                      .select(LC7_BANDS, STD_NAMES)
                      .map(cloudRemoval)
                      .map(GapFill))
            l5_col = (ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
                      .filterBounds(roi)
                      .filter(ee.Filter.calendarRange(year, year, 'year'))
                      .filter(ee.Filter.calendarRange(month, month, 'month'))
                      .map(applyScaleFactors)
                      .select(LC5_BANDS, STD_NAMES)
                      .map(cloudRemoval))
            landsat_col = ee.ImageCollection(l8_col.merge(l7_col).merge(l5_col)).sort("system:time_start")

            # Setting time information
            date = ee.Date.fromYMD(year, month, 1)

            # Use median composite monthly kNDVI data
            if landsat_col.size().getInfo() > 0:
                collection = landsat_col.map(clipImage)
                collection_ndvi = collection.map(addD2)
                sigma = collection_ndvi.select('sigma').reduce('mean')
                collection_ndvi = collection_ndvi.map(lambda image: addKNDVI(image, sigma))
                median_kndvi = collection_ndvi.select(['kndvi']).median()
                median_kndvi = (median_kndvi.toFloat()
                                .set('year', year)
                                .set('month', month)
                                .set('date', date.format("YYYY-MM-dd"))
                                .set('system:time_start', date.millis()))
                median_kndvi_collection = median_kndvi_collection.merge(ee.ImageCollection([median_kndvi]))
            else: # Create empty images for months with missing images.
                empty = ee.Image().clip(roi).rename(['kndvi'])
                empty_kndvi_img = (empty
                                .set('year',year)
                                .set('month',month)
                                .set('date', date.format("YYYY-MM-dd"))
                                .set('system:time_start', date.millis()))
                median_kndvi_collection = median_kndvi_collection.merge(ee.ImageCollection([empty_kndvi_img]))
                print("No images for " + str(year ) + "-" + str(month))
    return median_kndvi_collection

In [ ]:
# Synthesize data in batches to avoid exceeding memory limits.
median_kndvi_collection_1 = generate_median_kndvi_collection(2001, 2010)
median_kndvi_collection_2 = generate_median_kndvi_collection(2011, 2020)
merged_collection = median_kndvi_collection_1.merge(median_kndvi_collection_2)

In [ ]:
# Interpolation of missing images using Whittaker filter
import math

def extractBits(image, start, end, newName):
    pattern = 0
    for i in range(start, end+1):
        pattern += math.pow(2, i)
    return image.select([0], [newName]).bitwiseAnd(pattern).rightShift(start)

def getDifferenceMatrix(inputMatrix, order):
    rowCount = ee.Number(inputMatrix.length().get([0]))
    left = inputMatrix.slice(0,0,rowCount.subtract(1))
    right = inputMatrix.slice(0,1,rowCount)
    if order > 1:
        return getDifferenceMatrix(left.subtract(right), order-1)
    return left.subtract(right)

def unpack(arrayImage, imageIds, bands):
    def iter(item, icoll):
        def innerIter(innerItem, innerList):
            return ee.List(innerList).add(ee.String(item).cat("_").cat(ee.String(innerItem)))
        temp = bands.iterate(innerIter, ee.List([]))
        return ee.ImageCollection(icoll).merge(ee.ImageCollection(ee.Image(arrayImage).select(temp,bands).set("id",item)))
    imgcoll  = ee.ImageCollection(imageIds.iterate(iter, ee.ImageCollection([])))
    return imgcoll

def inverseLogRatio(image):
    bands = image.bandNames()
    t = image.get("system:time_start")
    ilrImage = ee.Image(100).divide(ee.Image(1).add(image.exp())).rename(bands)
    return ilrImage.set("system:time_start",t)

def whittakerSmoothing(imageCollection, isCompositional=False, lambda_=10):
    ic = imageCollection.map(lambda image: image.toFloat().set("system:time_start", image.get("system:time_start")))
    dimension = ic.size()
    identity_mat = ee.Array.identity(dimension)
    difference_mat = getDifferenceMatrix(identity_mat,3)
    difference_mat_transpose = difference_mat.transpose()
    lamda_difference_mat = difference_mat_transpose.multiply(lambda_)
    res_mat = lamda_difference_mat.matrixMultiply(difference_mat)
    hat_matrix = res_mat.add(identity_mat)
    original = ic
    properties = ee.List(ic.iterate(lambda image, list: ee.List(list).add(image.toDictionary()), []))
    time = ee.List(ic.iterate(lambda image, list: ee.List(list).add(image.get("system:time_start")), []))
    if isCompositional:
        ic = ic.map(lambda image: ee.Image(image).clamp(0.001,100-delta).divide(image).log().rename(bands).set("system:time_start", image.get("system:time_start")))
    arrayImage = original.toArray()
    coeffimage = ee.Image(hat_matrix)
    smoothImage = coeffimage.matrixSolve(arrayImage)
    idlist = ee.List(ic.iterate(lambda image, list: ee.List(list).add(image.id()), []))
    bandlist = ee.Image(ic.first()).bandNames()
    flatImage = smoothImage.arrayFlatten([idlist,bandlist])
    smoothCollection = ee.ImageCollection(unpack(flatImage, idlist, bandlist))
    if isCompositional:
        smoothCollection = smoothCollection.map(inverseLogRatio)
    newBandNames = bandlist.map(lambda band: ee.String(band).cat("_fitted"))
    smoothCollection = smoothCollection.map(lambda image: ee.Image(image).rename(newBandNames))
    dumbimg = arrayImage.arrayFlatten([idlist,bandlist])
    dumbcoll = ee.ImageCollection(unpack(dumbimg,idlist, bandlist))
    outCollection = dumbcoll.combine(smoothCollection)
    outCollectionProp = outCollection.iterate(lambda image,list: ee.List(list).add(image.set(properties.get(ee.List(list).size()))), [])
    outCollectionProp = outCollection.iterate(lambda image,list: ee.List(list).add(image.set("system:time_start",time.get(ee.List(list).size()))), [])
    residue_sq = smoothImage.subtract(arrayImage).pow(ee.Image(2)).divide(dimension)
    rmse_array = residue_sq.arrayReduce(ee.Reducer.sum(),[0]).pow(ee.Image(1/2))
    rmseImage = rmse_array.arrayFlatten([["rmse"],bandlist])
    return [ee.ImageCollection.fromImages(outCollectionProp), rmseImage]

def filterCol(img):
    newimg = img.copyProperties(img, img.propertyNames())
    return newimg

In [ ]:
latter_test = merged_collection.select('kndvi').map(filterCol)
latter_test = latter_test.map(lambda img: img.unmask(latter_test.mean()))
result_list = whittakerSmoothing(latter_test)

# Interpolated image
smoothed_collection = result_list[0]
# Root Mean Square Error
rmse_image = result_list[1]